# Estimating VaR in EURUSD from IV using ML and QR

## Data Processing

In [1]:
import pandas as pd

In [2]:
data_io = "EURUSD.xlsx"
df_raw = pd.read_excel(data_io, index_col=0)
df_raw.head()

,SPOT,FWD_1M,FWD_2M,FWD_3M,FWD_6M,FWD_1Y,IV_ATM_1W,IV_ATM_1M,IV_ATM_2M,IV_ATM_3M,...,RR_10D_6M,RR_10D_9M,RR_10D_1Y,RR_25D_1W,RR_25D_1M,RR_25D_2M,RR_25D_3M,RR_25D_6M,RR_25D_9M,RR_25D_1Y
DATE,,,,,,,,,,,,,,,,,,,,,
2010-07-01,0.80327,0.80320,0.80311,0.80298,0.80269,0.80224,14.750,14.10,14.200,14.350,...,-3.900,-4.775,-4.150,-0.15,-1.300,-1.775,-2.000,-2.300,-2.500,-2.600
2010-07-02,0.79706,0.79700,0.79690,0.79677,0.79649,0.79604,12.750,13.25,13.705,13.800,...,-4.150,-4.475,-4.775,0.25,-1.100,-1.625,-1.900,-2.200,-2.350,-2.475
2010-07-05,0.79782,0.79776,0.79766,0.79754,0.79725,0.79680,11.905,13.25,13.700,13.605,...,-4.225,-4.575,-4.850,-0.15,-1.150,-1.575,-1.875,-2.225,-2.400,-2.525
2010-07-06,0.79013,0.79007,0.78998,0.78991,0.78970,0.79680,12.750,12.80,12.950,13.200,...,-4.150,-4.375,-4.650,0.40,-0.975,-1.400,-1.775,-2.175,-2.275,-2.425
2010-07-07,0.79396,0.79393,0.79387,0.79380,0.79371,0.79352,11.705,12.70,12.900,13.255,...,-4.125,-4.450,-4.750,0.40,-0.875,-1.400,-1.800,-2.200,-2.350,-2.475


In [3]:
print(df_raw.columns.to_list())

['SPOT', 'FWD_1M', 'FWD_2M', 'FWD_3M', 'FWD_6M', 'FWD_1Y', 'IV_ATM_1W', 'IV_ATM_1M', 'IV_ATM_2M', 'IV_ATM_3M', 'IV_ATM_6M', 'IV_ATM_9M', 'IV_ATM_1Y', 'BT_10D_1W', 'BT_10D_1M', 'BT_10D_2M', 'BT_10D_3M', 'BT_10D_6M', 'BT_10D_9M', 'BT_10D_1Y', 'BT_25D_1W', 'BT_25D_1M', 'BT_25D_2M', 'BT_25D_3M', 'BT_25D_6M', 'BT_25D_9M', 'BT_25D_1Y', 'RR_10D_1W', 'RR_10D_1M', 'RR_10D_2M', 'RR_10D_3M', 'RR_10D_6M', 'RR_10D_9M', 'RR_10D_1Y', 'RR_25D_1W', 'RR_25D_1M', 'RR_25D_2M', 'RR_25D_3M', 'RR_25D_6M', 'RR_25D_9M', 'RR_25D_1Y']


In [4]:
# spot and 1 week
df_spot = df_raw[['SPOT', 'IV_ATM_1W', 'BT_10D_1W', 'BT_25D_1W',
                  'RR_10D_1W', 'RR_25D_1W']]
df_spot = df_spot.rename(
    columns={'SPOT':'SPOT', 'IV_ATM_1W':'IV_ATM',
             'BT_10D_1W':'BT_10D', 'BT_25D_1W':'BT_25D',
             'RR_10D_1W':'RR_10D', 'RR_25D_1W':'RR_25D'})
column_list = df_spot.columns.to_list()

df_spot.head()

,SPOT,IV_ATM,BT_10D,BT_25D,RR_10D,RR_25D
DATE,,,,,,
2010-07-01,0.80327,14.750,0.7063,0.4,-0.1,-0.15
2010-07-02,0.79706,12.750,0.7063,0.4,0.4,0.25
2010-07-05,0.79782,11.905,0.7063,0.4,0.4,-0.15
2010-07-06,0.79013,12.750,0.7063,0.4,0.4,0.40
2010-07-07,0.79396,11.705,0.7063,0.4,0.4,0.40


In [5]:
# spreads
# 1 month spread
df_1m = df_raw[['FWD_1M', 'IV_ATM_1M', 'BT_10D_1M', 'BT_25D_1M',
                'RR_10D_1M', 'RR_25D_1M']]
df_1m = df_1m.rename(
    columns={'FWD_1M':'SPOT', 'IV_ATM_1M':'IV_ATM',
             'BT_10D_1M':'BT_10D', 'BT_25D_1M':'BT_25D',
             'RR_10D_1M':'RR_10D', 'RR_25D_1M':'RR_25D'})
df_1m = df_1m - df_spot
df_1m = df_1m.rename(
    columns={item: f'{item}_1M' for item in column_list})

# 2 month spread
df_2m = df_raw[['FWD_2M', 'IV_ATM_2M', 'BT_10D_2M', 'BT_25D_2M',
                'RR_10D_2M', 'RR_25D_2M']]
df_2m = df_2m.rename(
    columns={'FWD_2M':'SPOT', 'IV_ATM_2M':'IV_ATM',
             'BT_10D_2M':'BT_10D', 'BT_25D_2M':'BT_25D',
             'RR_10D_2M':'RR_10D', 'RR_25D_2M':'RR_25D'})
df_2m = df_2m - df_spot
df_2m = df_2m.rename(
    columns={item: f'{item}_2M' for item in column_list})

# 3 month spread
df_3m = df_raw[['FWD_3M', 'IV_ATM_3M', 'BT_10D_3M', 'BT_25D_3M',
                'RR_10D_3M', 'RR_25D_3M']]
df_3m = df_3m.rename(
    columns={'FWD_3M':'SPOT', 'IV_ATM_3M':'IV_ATM',
             'BT_10D_3M':'BT_10D', 'BT_25D_3M':'BT_25D',
             'RR_10D_3M':'RR_10D', 'RR_25D_3M':'RR_25D'})
df_3m = df_3m - df_spot
df_3m = df_3m.rename(
    columns={item: f'{item}_3M' for item in column_list})

# 6 month spread
df_6m = df_raw[['FWD_6M', 'IV_ATM_6M', 'BT_10D_6M', 'BT_25D_6M',
                'RR_10D_6M', 'RR_25D_6M']]
df_6m = df_6m.rename(
    columns={'FWD_6M':'SPOT', 'IV_ATM_6M':'IV_ATM',
             'BT_10D_6M':'BT_10D', 'BT_25D_6M':'BT_25D',
             'RR_10D_6M':'RR_10D', 'RR_25D_6M':'RR_25D'})
df_6m = df_6m - df_spot
df_6m = df_6m.rename(
    columns={item: f'{item}_6M' for item in column_list})

# 9 month spread
df_9m = df_raw[['FWD_1Y', 'IV_ATM_9M', 'BT_10D_9M', 'BT_25D_9M',
                'RR_10D_9M', 'RR_25D_9M']]
df_9m = df_9m.rename(
    columns={'FWD_1Y':'SPOT', 'IV_ATM_9M':'IV_ATM',
             'BT_10D_9M':'BT_10D', 'BT_25D_9M':'BT_25D',
             'RR_10D_9M':'RR_10D', 'RR_25D_9M':'RR_25D'})
df_9m = df_9m - df_spot
df_9m = df_9m.rename(
    columns={item: f'{item}_9M' for item in column_list})
df_9m.drop('SPOT_9M', axis=1, inplace=True)

# 1 year spread
df_1y = df_raw[['FWD_1Y', 'IV_ATM_1Y', 'BT_10D_1Y', 'BT_25D_1Y',
                'RR_10D_1Y', 'RR_25D_1Y']]
df_1y = df_1y.rename(
    columns={'FWD_1Y':'SPOT', 'IV_ATM_1Y':'IV_ATM',
             'BT_10D_1Y':'BT_10D', 'BT_25D_1Y':'BT_25D',
             'RR_10D_1Y':'RR_10D', 'RR_25D_1Y':'RR_25D'})
df_1y = df_1y - df_spot
df_1y = df_1y.rename(
    columns={item: f'{item}_1Y' for item in column_list})

df_spread = pd.concat([df_spot, df_1m, df_2m, df_3m, df_9m, df_1y], axis=1)
df_spread.head()

,SPOT,IV_ATM,BT_10D,BT_25D,RR_10D,RR_25D,SPOT_1M,IV_ATM_1M,BT_10D_1M,BT_25D_1M,...,BT_10D_9M,BT_25D_9M,RR_10D_9M,RR_25D_9M,SPOT_1Y,IV_ATM_1Y,BT_10D_1Y,BT_25D_1Y,RR_10D_1Y,RR_25D_1Y
DATE,,,,,,,,,,,,,,,,,,,,,
2010-07-01,0.80327,14.750,0.7063,0.4,-0.1,-0.15,-0.00007,-0.650,0.4312,-0.0750,...,1.5062,0.3000,-4.675,-2.350,-0.00103,0.000,1.8812,0.4250,-4.050,-2.450
2010-07-02,0.79706,12.750,0.7063,0.4,0.4,0.25,-0.00006,0.500,0.4187,-0.0750,...,1.5062,0.2875,-4.875,-2.600,-0.00102,1.600,1.8687,0.4125,-5.175,-2.725
2010-07-05,0.79782,11.905,0.7063,0.4,0.4,-0.15,-0.00006,1.345,0.4187,-0.0875,...,1.4937,0.2875,-4.975,-2.250,-0.00102,2.365,1.8562,0.4125,-5.250,-2.375
2010-07-06,0.79013,12.750,0.7063,0.4,0.4,0.40,-0.00006,0.050,0.4187,-0.0875,...,1.4937,0.2750,-4.775,-2.675,0.00667,1.200,1.8562,0.4000,-5.050,-2.825
2010-07-07,0.79396,11.705,0.7063,0.4,0.4,0.40,-0.00003,0.995,0.3812,-0.1000,...,1.4937,0.2750,-4.850,-2.750,-0.00044,2.195,1.8562,0.4000,-5.150,-2.875


In [6]:
df_spot.to_hdf('df_spot.h5', key='df', mode='w')
df_spread.to_hdf('df_spread.h5', key='df', mode='w')